In [1]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential
tf.compat.v1.disable_eager_execution()
print(tf.compat.v1.get_default_graph())
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
lyrics = open("Lyrics_Counterparts.txt", encoding="utf-8").read().lower()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(lyrics)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[17, 5, 8, 9, 3]]

In [5]:
max_id = len(tokenizer.word_index) #number of distinct characters in text
dataset_size = tokenizer.document_count #total number of characters
print(max_id, " ", dataset_size)

52   71707


In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([lyrics])) - 1
#encodes the full text so each character is represented by its ID

In [7]:
#splitting dataset for training/test/validation
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
#convert long string of characters into windows for training(batches)
n_steps = 75
window_length = n_steps + 1 #target character = input shifted 1 char ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
#flatten the dataset for training
dataset = dataset.flat_map(lambda window: window.batch(window_length))
#this gives us a single tensor for each window

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [12]:
dataset = dataset.prefetch(1)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                    dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on None steps
Epoch 1/20
2015/2015 [==============================] - 447s 222ms/step - loss: 1.9389
Epoch 2/20
2015/2015 [==============================] - 438s 217ms/step - loss: 1.5561
Epoch 3/20
 710/2015 [=========>....................] - ETA: 4:53 - loss: 1.4988

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["Selfishly I s"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess[(text)]
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/ temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
#this function picks the next character randomly, with a probability equal to the estimated probability,using tf.random.categorical
#categorical() samples random class indices, given the class probs (logits)
#temperature is a variables that controls the diversity of the characters generated
# a near-zero temperature will output characters with high probabilities
# a high temperature will give all characters equal probability

In [ ]:
#this function repeatedly calls the above function to generate text
def complete_text(text, n_chars=75, temperature=1):
    for _ in range(n_chars):
        text += next_char, temperature)
    return text